In [2]:
import psycopg2 as pg

conn = pg.connect(database="NAaaS",user="postgres", password="1234", host="127.0.0.1", port="8008")
	
cursor = conn.cursor()

# cursor.execute("select * from news")

# print(cursor.fetchone())

In [4]:
import pandas as pd
df = pd.read_csv("../Data_Generator/Results.csv")
# df["Link"] =
del df["CreationDate"]
# print(df.columns)
# df.head()
conn = pg.connect(database="NAaaS",user="postgres", password="1234", host="127.0.0.1", port="8008")
cursor = conn.cursor()

for data in df.itertuples():
    # print(data[5])
    # break
    cursor.execute("Select name from province where name=%s",
                   (str(data[7]).upper(),))
    province = cursor.fetchone()
    
    if province:
        cursor.execute("Insert into NEWS(header, summary, details, focus_time, focus_location, link, category, province) VALUES( %s, %s, %s, %s, %s, %s, %s, %s);",
                       (data[2], data[3], data[4], data[1], province[0], data[5], data[6],  province[0]))
        # print(conn.status)

    else:
        # break
        cursor.execute(
            "Select name, province from district where name=%s", (str(data[7]).upper(),))
        district = cursor.fetchone()
        if district:
            cursor.execute("Insert into NEWS(header, summary, details, focus_time, focus_location, link, category, province, district) VALUES( %s, %s, %s, %s, %s, %s, %s, %s, %s);",
                       (data[2], data[3], data[4], data[1], district[0], data[5], data[6],  district[1], district[0]))
        else:
            cursor.execute(
                "Select name, district from tehsil where name=%s", (str(data[7]).upper(),))
            tehsil = cursor.fetchone()
            if tehsil:
                cursor.execute(
                    "Select name, province from district where name=%s", (tehsil[1],))
                district = cursor.fetchone()
                if district:
                    cursor.execute("Insert into NEWS(header, summary, details, focus_time, focus_location, link, category, province, district, tehsil) VALUES( %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);",
                       (data[2], data[3], data[4], data[1], tehsil[0], data[5], data[6],  district[1], district[0], tehsil[0]))


In [5]:
conn.commit()

# cursor.execute("select tehsil from news")
# cursor.fetchall()